# Image Compression using Modified Singular Value Decomposition

### SVD and Image compression

Now we will explore how to apply Singular Value Decomposition of a matrix to the problem of image compression. SVD decomposes a rectangular matrix $M$ to a three parts.
$M=U\Sigma V^T$  -
- $U$ - matrix of left singular vectors in the columns
- $\Sigma$ - diagonal matrix with singular values
- $V$ - matrix of right singular vectors in the columns

SVD in effect involves reconstructing the original matrix as a linear combination of several rank one matrices. A rank one matrix can be expressed as a outer product of two column vectors. 

$M=\sigma_1u_1v_1^T+\sigma_2u_2v_2^T+\sigma_3u_3v_3^T+\sigma_3u_3v_3^T+....$ . 
A matrix of rank r will have r terms of these.

Here $\sigma_1,\sigma_2,\sigma_3 ...$ are singular values. $u_1,u_2,u_3 ...$ and $v_1,v_2,v_3 ...$ are left and right singular vectors respectively.

Image compression using SVD involves taking advantage of the fact that very few of the singular values are large. Although images from the real world are of full rank, they have low effective rank which means that only few of the singular values of the SVD of images will be large.

### skimage image processing library

We will use skimage image processing library (from sci-kit family of packages) for working with images in python. skimage has a module called data which makes available a set of images for exploration.  We will load some images and convert them into a gray scale format. These images are stored in a python dict object gray_images.

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from ipywidgets import interact, interactive, interact_manual
from skimage import data
from PIL import Image
import cv2
from scipy.misc import imsave
import sys
import imageio
from skimage import img_as_ubyte,img_as_float, img_as_int
from numpy.linalg import svd

In [9]:
#This line is required to display visualizations in the browser
%matplotlib inline

### svd in python
We will use ```numpy.linalg``` library's ```svd``` function to compute svd of a matrix in python. The svd function returns U,s,V .
   - U has left singular vectors in the columns
   - s is rank 1 numpy array with singular values
   - V has right singular vectors in the rows -equivalent to $V^T$ in traditional linear algebra literature
   
The reconstructed approximation of the original matrix is done using a subset of singular vectors as below in the ```compress_svd``` function . We use numpy array slicing to select k singular vectors and values.  Instead of storing $m\times n$ values for the original image, we can now store $k(m+n)+k$ values

           reconst_matrix = np.dot(U[:,:k],np.dot(np.diag(s[:k]),V[:k,:]))
           

In [10]:
def compress_svd(image,k):
    """
    Perform svd decomposition and truncated (using k singular values/vectors) reconstruction
    returns
    --------
      reconstructed matrix reconst_matrix, array of singular values s
    """
    U,s,V = svd(image,full_matrices=False)
    reconst_matrix = np.dot(U[:,:k],np.dot(np.diag(s[:k]),V[:k,:]))
   
    return reconst_matrix,s

### Load color images

In [11]:
color_images = {                                                                   # Position of Original Image
    "cat":img_as_float(data.chelsea()),                                            # L
    "astro":img_as_float(data.astronaut()),                                        # R
    "coffee":img_as_float(data.coffee()),                                          # L
    "hubble":img_as_float(data.hubble_deep_field()),                               # R
    "chemistry":img_as_float(data.immunohistochemistry()),                         # L
    "rocket":img_as_float(data.rocket()),                                          # R
    "boats":img_as_float(mpimg.imread('images/boats.png')),                        # L
    "canyon":img_as_float(mpimg.imread('images/canyon.png')),                      # R
    "cliff":img_as_float(mpimg.imread('images/cliff.png')),                        # L
    "countryside":img_as_float(mpimg.imread('images/countryside.png')),            # R
    "cows":img_as_float(mpimg.imread('images/cows.png')),                          # L
    "egypt":img_as_float(mpimg.imread('images/egypt.png')),                        # R
    "mass":img_as_float(mpimg.imread('images/mass.png')),                          # L
    "palawan":img_as_float(mpimg.imread('images/palawan.png')),                    # R
    "rock_formations":img_as_float(mpimg.imread('images/rock_formations.png')),    # L
    "sphinx":img_as_float(mpimg.imread('images/sphinx.png')),                      # R
    "temple":img_as_float(mpimg.imread('images/temple.png')),                      # L
    "tourists":img_as_float(mpimg.imread('images/tourists.png')),                  # R
    "mug":img_as_float(mpimg.imread('images/mug.png')),                            # L
    "bathtub":img_as_float(mpimg.imread('images/bathtub.png')),                    # R
    "building":img_as_float(mpimg.imread('images/building.png')),                  # L
    "chipmunk":img_as_float(mpimg.imread('images/chipmunk.png')),                  # R
    "wall":img_as_float(mpimg.imread('images/wall.png')),                          # L
    "bed":img_as_float(mpimg.imread('images/bed.png'))                             # R
}

### Compress color images

Color images are represented in python as 3 dimensional numpy arrays - the third dimension to represent the color values (red,green blue). However, svd method is applicable to two dimensional matrices. So we have to find a way to convert the 3 dimensional array to 2 dimensional arrays, apply svd and reconstruct it back as a 3 dimensional array . There are two ways to do it. We will show both these methods below .
   - reshape method
   - Layer method


Here is the interactive widget to explore image compression of color images using the reshape method. By dragging the slider to vary $k$, observe how image quality varies. Also, we can explore different images by selecting through the drop down widget.

### Layers method to compress color images
In the function ```compress_show_color_images_layer```, we treat a color image as a stack of 3 seperate two dimensional images  (Red,blue and green layers) . We apply the truncated svd reconstruction on each two dimensional layer seperately.

```image_reconst_layers = [compress_svd(image[:,:,i],k)[0] for i in range(3)]```

And we put back the reconstructed layers together.

```
image_reconst = np.zeros(image.shape)
for i in range(3):
   image_reconst[:,:,i] = image_reconst_layers[i]
```


   

In [12]:
def compute_k(height, width):
    avg = (height + width) / 2
    k = (2 * (avg / 100) + 1) * 10
    return k

In [13]:
def compress_show_color_images_layer(img_name):
    """
     compress and display the reconstructed color image using the layer method 
    """

    index = list(color_images.keys()).index(img_name)
    print('index = ' + str(index))

    image = color_images[img_name]
    original_image = image
    original_shape = image.shape
    k = int(compute_k(original_shape[0],original_shape[1]))
    print('computed k = ' + str(k))
    image_reconst_layers = [compress_svd(image[:,:,i],k)[0] for i in range(2)]
    image_reconst = np.zeros(image.shape)
    nrows = original_shape[0]
    ncols = original_shape[1]
    nchans = original_shape[2]
    print("width = " + str(ncols) + ", height = " + str(nrows) + ", channels = " + str(nchans))
    image_reconst_layers.append(image[:,:,2])
    for i in range(nrows):
        for j in range(ncols):
            for c in range(nchans):
                val = image_reconst_layers[c][i][j]
                if val < 0.0:
                    val = 0.0
                elif val > 1.0:
                    val = 1.0
                image_reconst[i][j][c] = val
    
    fig,axes = plt.subplots(1,2,figsize=(8,5))
    compression_ratio =100.0*2* (k*(original_shape[0] + original_shape[1])+k)/(original_shape[0]*original_shape[1]*original_shape[2])
    axes[0].set_title("Compressed Image\ncompression ratio = {:.2f}".format(compression_ratio)+"%")
    axes[0].imshow(image_reconst)
    axes[1].set_title("Original Image")
    axes[1].imshow(original_image)    
    fig.tight_layout()
    
    if index % 2 == 0:
        vis = np.concatenate((original_image, image_reconst), axis=1)
    else:
        vis = np.concatenate((image_reconst, original_image), axis=1)
    imsave('images/compressed/' + img_name + '_compressed.png', vis)
    imsave('images/compressed/unstacked/' + img_name + '_compressed.png', image_reconst)

Here is the widget to explore layers method of compressing color images.

In [14]:
interact(compress_show_color_images_layer,img_name=list(color_images.keys()));

interactive(children=(Dropdown(description='img_name', options=('cat', 'astro', 'coffee', 'hubble', 'chemistry…